In [1]:
"""Tutorial: CEPA0 and CCD"""

__author__    = "Adam S. Abbott"
__credit__    = ["Adam S. Abbott", "Justin M. Turney"]

__copyright__ = "(c) 2014-2017, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-05-23"

# Introduction
In this tutorial, we will code up the coupled-electron pair approximation (CEPA0) and coupled-cluster doubles (CCD) using our spin-orbital framework covered in the previous tutorial.


### I. Coupled Cluster Theory

In single-reference coupled cluster theory, dynamic correlation is acquired by operating an exponential of cluster operators on some reference determinant, such as a Hartree-Fock wavefunction, to obtain a new wavefunction:

\begin{equation}
\mid \Psi_{CC} \rangle = exp(\hat{T}) \mid \Phi \rangle 
\end{equation}

where $\hat{T} = T_1 + T_2 + ... + T_n$ is the sum of "cluster operators" which which act on our reference to excite electrons from occupied ($i, j, k$...) to virtual ($a, b, c$...) orbitals. In the second quantization, these cluster operators are expressed as:

\begin{equation}
T_n = \left(\frac{1}{k!}\right)^2 \sum_{\substack{i_1 \ldots i_k \\ a_1 \ldots a_k }} t_{i_1 \ldots i_k}^{a_1 \ldots a_k}   a_{a_1}^{\dagger} \ldots a_{a_k}^{\dagger} a_{i_k} \ldots a_{i_1}
\end{equation}

where $t$ is the $t$-amplitude, and $a^{\dagger}$ and $a$ are creation and annihilation operators.

### II. Coupled Cluster Doubles
For CCD, we only include the doubles cluster operator:

\begin{equation}
\mid \Psi_{CCD} \rangle = exp(\hat{T_2}) \mid \Phi \rangle
\end{equation}

I will spare you the details, but if one projects the CCD Schr&ouml;dinger equation on the left by a doubly-excited reference determinant $ \langle\Phi_{ij}^{ab}\mid $, evaluates the diagrams, and rearranges the terms, one obtains the equation for the CCD amplitudes: 

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \overline{g}_{ab}^{ij} + \tfrac{1}{2} \overline{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \overline{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \overline{g}_{ak}^{ic} t_{bc}^{jk} - \tfrac{1}{2}\hat{P}_{(a \space / \space b)} \overline{g}_{kl}^{cd} t_{ac}^{ij} t_{bd}^{kl} - \tfrac{1}{2}  \hat{P}^{(i \space / \space j)} \overline{g}_{kl}^{cd} t_{ab}^{ik} t_{cd}^{jl} + \tfrac{1}{4} \overline{g}_{kl}^{cd} t_{cd}^{ij} t_{ab}^{kl} +  \hat{P}^{(i \space / \space j)} \overline{g}_{kl}^{cd} t_{ac}^{ik} t_{bd}^{jl} \right)
\end{equation}

where $(\mathcal{E}_{ab}^{ij})^{-1}$ is the orbital energy denominator, more familiarly known as $\frac{1}{\epsilon_i + \epsilon_j - \epsilon_a - \epsilon_b}$ and $\overline{g}_{pq}^{rs}$ is the antisymmetrized two-electron integral in physicist's notation $\langle pq \mid\mid rs \rangle$. $\hat{P}$ is the *antisymmetric permutation operator*. This operator acts on a term to produce the sum of the permutations of the indicated indices, with an appropriate sign factor. It's effect is best illustrated by an example. Consider the fourth term, which is really 4 terms in one. $\hat{P}_{(a \space / \space b)}^{(i \space / \space j)}$ produces: 
1. The original: $ \quad \quad \quad \quad \quad \quad \space \overline{g}_{ak}^{ic} t_{bc}^{jk} $
2. Permuation of $a$ and $b$: $ \quad \quad - \overline{g}_{bk}^{ic} t_{ac}^{jk} $
3. Permuation of $i$ and $j$: $ \quad \space \quad - \overline{g}_{ak}^{jc} t_{bc}^{ik} $
4. Permuation of $a$ and $b$, $i$ and $j$: $ \space \overline{g}_{bk}^{jc} t_{ac}^{ik} $

Note that each permutation adds a sign change. This shorthand notation keeps the equation in a more manageable form. 

The corresponding energy expression can be obtained in somewhat similar way, this time by projecting CCD Schr&ouml;dinger equation on the left by $\langle \Phi \mid$. This yields:

\begin{equation}
E_{CCD} = \tfrac{1}{4} \overline{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

Since the $t$ amplitude is dependent on the $t$ amplitude, we must iteratively solve for it until the energy reaches some convergence threshold.

### III. Retrieving MP2 and CEPA0 from the CCD equations
It is interesting to note that if we only consider the first term of the expression for the doubles amplitude $t_{ab}^{ij}$ and plug it into the energy expression, we obtain the MP2 energy expression:

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \overline{g}_{ab}^{ij} 
\end{equation}

\begin{equation}
E_{MP2} = \tfrac{1}{4}  \overline{g}_{ab}^{ij} t_{ij}^{ab}  = \tfrac{1}{4} \overline{g}_{ab}^{ij} \overline{g}_{ij}^{ab}  (\mathcal{E}_{ab}^{ij})^{-1}
\end{equation}

Furthermore, if we leave out the quadratic terms in the CCD amplitude equation (terms containing two $t$-amplitudes), we obtain the coupled electron-pair approximation (CEPA0):
\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \overline{g}_{ab}^{ij} + \tfrac{1}{2} \overline{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \overline{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \overline{g}_{ak}^{ic} t_{bc}^{jk} \right)
\end{equation}

and the CEPA0 energy expression is identical:

\begin{equation}
E_{CEPA0} = \tfrac{1}{4} \overline{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

Using our spin-orbital setup for the MO coefficients, orbital energies, and two-electron integrals used in the previous tutorial, we are equipped to program the expressions for the CEPA0 and CCD correlation energy directly.

### CEPA0 and CCD
As usual, we import Psi4 and Numpy, and set the appropriate options. 

In [ ]:
# ==> Import statements & Global Options <==
import psi4
import numpy as np

psi4.set_memory(int(2e9))
numpy_memory = 2
psi4.core.set_output_file('output.dat', False)

In [ ]:
# ==> Molecule & Psi4 Options Definitions <==
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")



psi4.set_options({'basis':        '6-31g',
                  'scf_type':     'pk',
                  'reference':    'rhf'
                  'mp2_type':     'conv',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

Note that since we are using a spin-orbital setup, we are free to use any Hartree-Fock reference we want. Here we choose RHF. For convenience, we let Psi4 take care of the Hartree-Fock procedure, and return the wavefunction object.

In [ ]:
# Get the SCF wavefunction & energies
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)

Load in information about the basis set and orbitals using MintsHelper and the wavefunction:

In [ ]:
mints = psi4.core.MintsHelper(scf_wfn.basisset())
nbf = mints.nbf()           # number of basis functions
nso = 2 * nbf               # number of spin orbitals
nalpha = scf_wfn.nalpha()   # number of alpha electrons
nbeta = scf_wfn.nbeta()     # number of beta electrons
nocc = nalpha + nbeta       # number of occupied orbitals
nvirt = 2 * nbf - nocc      # number of virtual orbitals

Spin-block our MO coefficients and two-electron integrals, just like in the spin-orbital MP2 code:

In [ ]:
Ca = np.asarray(scf_wfn.Ca())
Cb = np.asarray(scf_wfn.Cb())
C = np.block([
             [      Ca           ,   np.zeros_like(Cb) ],
             [np.zeros_like(Ca)  ,          Cb         ]
            ])

# Result: | Ca  0 |
#         | 0   Cb|


In [1]:
# Get the two electron integrals using MintsHelper
I = np.asarray(mints.ao_eri())

# Function that spin blocks two-electron integrals
# Using np.kron, we project I into the space of the 2x2 identity, tranpose the result
# and project into the space of the 2x2 identity again. This doubles the size of each axis.
# The result is our two electron integral tensor in the spin orbital form.
def spin_block_tei(I):
    identity = np.eye(2)
    I = np.kron(identity, I)
    return np.kron(identity, I.T)

I_spinblock = spin_block_tei(I)


NameError: name 'np' is not defined

Convert two-electron integrals to antisymmetrized physicist's notation:

In [ ]:
# Converts chemist's notation to physicist's notation, and antisymmetrize
# (pq | rs) ---> <pr | qs>
# Physicist's notation
tmp = I_spinblock.transpose(0, 2, 1, 3)
# Antisymmetrize:
# <pr||qs> = <pr | qs> - <pr | sq>
gao = tmp - tmp.transpose(0, 1, 3, 2)


Obtain the orbital energies, append them, and sort the columns of our MO coefficient matrix according to the increasing order of orbital energies. 

In [ ]:
# Get orbital energies 
eps_a = np.asarray(scf_wfn.epsilon_a())
eps_b = np.asarray(scf_wfn.epsilon_b())
eps = np.append(eps_a, eps_b)

# Before sorting the orbital energies, we can use their current arrangement to sort the columns
# of C. Currently, each element i of eps corresponds to the column i of C, but we want both
# eps and columns of C to be in increasing order of orbital energies

# Sort the columns of C according to the order of increasing orbital energies 
C = C[:, eps.argsort()] 

# Sort orbital energies in increasing order
eps = np.sort(eps) 


Finally, we transform our two-electron integrals to the MO basis. Here, we denote the integrals as `gmo` to differentiate from the chemist's notation integrals `I_mo`.

In [ ]:
# Transform gao, which is the spin-blocked 4d array of physicist's notation, 
# antisymmetric two-electron integrals, into the MO basis using MO coefficients 
gmo = np.einsum('pQRS, pP -> PQRS',
      np.einsum('pqRS, qQ -> pQRS',
      np.einsum('pqrS, rR -> pqRS',
      np.einsum('pqrs, sS -> pqrS', gao, C), C), C), C)


Construct the 4-dimensional array of orbital energy denominators:

In [ ]:
# Define slices, create 4 dimensional orbital energy denominator tensor
n = np.newaxis
o = slice(None, nocc)
v = slice(nocc, None)
e_abij = 1 / (-eps[v, n, n, n] - eps[n, v, n, n] + eps[n, n, o, n] + eps[n, n, n, o])

We now have everything we need to construct our $t$ amplitudes and iteratively solve for our CEPA0 and CCD energy. To build the $t$ amplitudes, we first construct an empty 4-dimensional array to store them. 

In [1]:
# Create space to store t amplitudes
t_amp = np.zeros((nvirt, nvirt, nocc, nocc))


NameError: name 'np' is not defined

 First we will program CEPA0. Recall the expression for the $t$ amplitudes:

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \overline{g}_{ab}^{ij} + \tfrac{1}{2} \overline{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \overline{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \overline{g}_{ak}^{ic} t_{bc}^{jk} \right)
\end{equation}

These terms translate naturally into code using Numpy's einsum function. To access only the occupied and virtual indices of `gmo` we use our slices defined above. The permutation operator terms can be easily obtained by transposing the original result accordingly. To construct each iteration's $t$ amplitude:  

~~~python
mp2    = gmo[v, v, o, o]
cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
cepa3b = -cepa3a.transpose(1, 0, 2, 3)
cepa3c = -cepa3a.transpose(0, 1, 3, 2)
cepa3d =  cepa3a.transpose(1, 0, 3, 2)
cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3)
~~~

To evaluate the energy, $E_{CEPA0} = \tfrac{1}{4} \overline{g}_{ij}^{ab} t_{ab}^{ij}$,

~~~python
E_CEPA0 = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
~~~

Putting it all together, we initialize the energy, set the max iterations, and iterate the energy until it falls below our energy convergence criterion:

In [ ]:
# Create space to store t amplitudes
t_amp = np.zeros((nvirt, nvirt, nocc, nocc))

# Initialize energy
E_CEPA0 = 0.0

MAXITER = 50

for cc_iter in range(MAXITER + 1):
    E_old = E_CEPA0
    
    # Collect terms
    mp2    = gmo[v, v, o, o]
    cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
    cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
    cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
    cepa3b = -cepa3a.transpose(1, 0, 2, 3)
    cepa3c = -cepa3a.transpose(0, 1, 3, 2)
    cepa3d =  cepa3a.transpose(1, 0, 3, 2)
    cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

    # Update t amplitude
    t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3)

    # Evaluate Energy
    E_CEPA0 = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
    t_amp = t_amp_new
    dE = E_CEPA0 - E_old
    print('CEPA0 Iteration %3d: Energy = %4.12f dE = %1.5E' % (cc_iter, E_CEPA0, dE))

    if abs(dE) < 1.e-8:
        print("\nCEPA0 Iterations have converged!")
        break

    if (cc_iter == MAXITER):
        psi4.core.clean()
        raise Exception("\nMaximum number of iterations exceeded.")

print('\nCEPA0 Correlation Energy: %5.15f' % (E_CEPA0))
print('CEPA0 Total Energy: %5.15f' % (E_CEPA0 + scf_e))

Since t_amp is initialized to zero, the very first iteration should be the MP2 correlation energy. We can check the final CEPA0 energy with Psi4. The method is called `lccd`, or linear CCD, since CEPA0 omits the terms with two cluster amplitudes.

In [ ]:
psi4.driver.p4util.compare_values(psi4.energy('lccd'), E_CEPA0 + scf_e, 6, 'CEPA0 Energy')

To code up full CCD, we only have to add in the last four terms in our expression for the $t$ amplitude: 

\begin{equation}
t_{ab}^{ij} = (\mathcal{E}_{ab}^{ij})^{-1} \left( \overline{g}_{ab}^{ij} + \tfrac{1}{2} \overline{g}_{ab}^{cd} t_{cd}^{ij} + \tfrac{1}{2} \overline{g}_{kl}^{ij} t_{ab}^{kl}  + \hat{P}_{(a \space / \space b)}^{(i \space / \space j)} \overline{g}_{ak}^{ic} t_{bc}^{jk} - \underline{\tfrac{1}{2}\hat{P}_{(a \space / \space b)} \overline{g}_{kl}^{cd} t_{ac}^{ij} t_{bd}^{kl} - \tfrac{1}{2}  \hat{P}^{(i \space / \space j)} \overline{g}_{kl}^{cd} t_{ab}^{ik} t_{cd}^{jl} + \tfrac{1}{4} \overline{g}_{kl}^{cd} t_{cd}^{ij} t_{ab}^{kl} +  \hat{P}^{(i \space / \space j)} \overline{g}_{kl}^{cd} t_{ac}^{ik} t_{bd}^{jl}} \right)
\end{equation}

which we readily translate into einsum's:

~~~python
ccd1a  = np.einsum('klcd, acij, bdkl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd1b  = -ccd1a.transpose(1, 0, 2, 3)
ccd1   = -(1 / 2) * (ccd1a + ccd1b)

ccd2a  = np.einsum('klcd, abik, cdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd2b  = -ccd2a.transpose(0, 1, 3, 2)
ccd2   = -(1 / 2) * (ccd2a + ccd2b)

ccd3   =  (1 / 4) * np.einsum('klcd, cdij, abkl -> abij', gmo[o, o, v, v], t_amp, t_amp)

ccd4a  = np.einsum('klcd, acik, bdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
ccd4b  = -ccd4a.transpose(0, 1, 3, 2)
ccd4   = (ccd4a + ccd4b)
~~~

and the energy expression is identical:
\begin{equation}
E_{CCD } = \tfrac{1}{4} \overline{g}_{ij}^{ab} t_{ab}^{ij}
\end{equation}

Adding the above terms to our CEPA0 code obtains the CCD correlation energy (may take a minute or two):

In [ ]:
for cc_iter in range(1, MAXITER + 1):
    E_old = E_CCD

    # Collect terms
    mp2    = gmo[v, v, o, o]
    cepa1  = (1 / 2) * np.einsum('abcd, cdij -> abij', gmo[v, v, v, v], t_amp)
    cepa2  = (1 / 2) * np.einsum('klij, abkl -> abij', gmo[o, o, o, o], t_amp)
    cepa3a = np.einsum('akic, bcjk -> abij', gmo[v, o, o, v], t_amp)
    cepa3b = -cepa3a.transpose(1, 0, 2, 3)
    cepa3c = -cepa3a.transpose(0, 1, 3, 2)
    cepa3d =  cepa3a.transpose(1, 0, 3, 2)
    cepa3  =  cepa3a + cepa3b + cepa3c + cepa3d

    ccd1a  = np.einsum('klcd, acij, bdkl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd1b  = -ccd1a.transpose(1, 0, 2, 3)
    ccd1   = -(1 / 2) * (ccd1a + ccd1b)

    ccd2a  = np.einsum('klcd, abik, cdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd2b  = -ccd2a.transpose(0, 1, 3, 2)
    ccd2   = -(1 / 2) * (ccd2a + ccd2b)

    ccd3   =  (1 / 4) * np.einsum('klcd, cdij, abkl -> abij', gmo[o, o, v, v], t_amp, t_amp)

    ccd4a  = np.einsum('klcd, acik, bdjl -> abij', gmo[o, o, v, v], t_amp, t_amp)
    ccd4b  = -ccd4a.transpose(0, 1, 3, 2)
    ccd4   = (ccd4a + ccd4b)

    # Update Amplitude
    t_amp_new = e_abij * (mp2 + cepa1 + cepa2 + cepa3 + ccd1 + ccd2 + ccd3 + ccd4)

    # Evaluate Energy
    E_CCD = (1 / 4) * np.einsum('ijab, abij ->', gmo[o, o, v, v], t_amp_new)
    t_amp = t_amp_new
    dE = E_CCD - E_old
    print('CCD Iteration %3d: Energy = %4.12f dE = %1.5E' % (cc_iter, E_CCD, dE))

    if abs(dE) < 1.e-8:
        print("\nCCD Iterations have converged!")
        break

    if (cc_iter == MAXITER):
        psi4.core.clean()
        raise Exception("\nMaximum number of iterations exceeded.")

print('\nCCD Correlation Energy: %5.15f' % (E_CCD))
print('CCD Total Energy: %5.15f' % (E_CCD + scf_e))


Unfortunately, Psi4 does not have a CCD code to compare this to. However, Psi4 does have Bruekner CCD, an orbital-optimized variant of CCD. We can qualitatively compare our energies to this energy. The Bruekner-CCD energy should be a little lower than our CCD due to the orbital optimization procedure.

In [ ]:
psi4_bccd = psi4.energy('bccd', ref_wfn = scf_wfn)
print('\nPsi4 BCCD Correlation Energy:     ', psi4_bccd - scf_e)
print('Psi4 BCCD Total Energy:     ', psi4_bccd)

## References

1. Review of coupled-cluster theory, included diagrammatic derivations of the CCD equations:
	> [[Bartlett and Musial:2007](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.79.291)] Rodney J. Bartlett and Monika Musial,  "Coupled-cluster theory in quantum chemistry" *Rev. Mod. Phys.* **79**, 291 (2007)
2. Useful Notes on Kutzelnigg-Mukherjee Notation: 
    > [Copan: KM Notation] A. V. Copan, "Kutzelnigg-Mukherjee Tensor Notation" Accessed with https://github.com/avcopan/chem-8950/blob/master/3q-1h-kutzelnigg-mukherjee.pdf
    
